In [1]:
import os
print(os.getcwd())

/home/hiimbias/Workspace/aiagent/test


In [2]:
os.chdir('/home/hiimbias/Workspace/aiagent')
print(os.getcwd())  # Verify the change

/home/hiimbias/Workspace/aiagent


In [3]:
from modules.openai.openai_executor import OpenAIExecutor
from core.utils.tool_wrapper import tool
from pydantic import BaseModel
from typing import Annotated
import openai
import os
from modules.tools.get_weather import get_weather_tool

In [4]:
@tool(
    description="Calculate the sum of 2 numbers", # Define the tool description
)
def calculate(a: Annotated[int, "a"], b: Annotated[int, "b"]) -> int:
    """Calculate the sum of two numbers"""
    return a + b

In [5]:
calculate.schema

{'type': 'function',
 'function': {'name': 'calculate',
  'description': 'Calculate the sum of two numbers',
  'strict': True,
  'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
    'b': {'title': 'B', 'type': 'integer'}},
   'required': ['a', 'b'],
   'type': 'object',
   'additionalProperties': False}}}

In [6]:
# Initialize OpenAI client
client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

# Initialize LLM service
executor = OpenAIExecutor(
    client=client,
    model="gpt-4o-mini",
    system_message="""
    You are a helpful assistant that can answer questions and help with tasks.
    You are also able to use tools to get information.
    """,
    tools=[get_weather_tool, calculate],
    temperature=0.5,
    max_tokens=100,
    top_p=1.0,
)

generator = executor.execute(
    messages=[
        {"role": "user", "content": "What's the weather like in New York?"}
    ]
)

for response in generator:
    print(response)

role='assistant' content=None tool_calls=[{'id': 'call_cXigMXasfowOZnz3Y9jTkmuu', 'type': 'function', 'function': {'arguments': '{"mode":"current","location":"New York","days":1}', 'name': 'get_weather_tool'}}] tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=197, completion_tokens=25, total_tokens=222, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))
role='tool' content=None tool_calls=None tool_results=[ToolCallResult(tool_name='get_weather_tool', result=WeatherResponse(location='New York', current_weather=CurrentWeather(date='2025-05-13 23:30', temp=15.4, chance_of_rain=75.0, air_quality=2, humidity=93.0, condition='Light rain'), forecast=None, error=None))] refusal=None audio=None usage=None
role='assistant' content='The current weather in New York is as follows:\n\n- **Temperature:**

In [7]:
executor.get_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-05-14 10:31:30\n\n        \n        '}]

In [8]:
executor.clear_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-05-14 10:31:30\n\n        \n        '}]

In [ ]:
while True:
    user_input = input("User: ")
    print(f"User: {user_input}")
    
    if user_input.lower() == "exit":
        break

    generator = executor.execute(
        messages=[
            {"role": "user", "content": user_input}
        ]
    )

    for response in generator:
        if response.content is not None:
            print(f"AI: {response.content}")
        if response.tool_calls:
            [print(f"Tool Calls: {tool_call}") for tool_call in response.tool_calls]

In [ ]:
executor.get_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-05-03 00:00:28\n\n        \n        '}]